In [ ]:
%matplotlib inline
%pylab inline

import menpo.io as mio
from menpo.feature import no_op, fast_dsift
from menpo.landmark import labeller, ibug_face_66
from menpo.visualize import visualize_images, print_dynamic

from menpofit.clm.patch_experts import (
    build_mccf_ensemble, build_lsvm_ensemble)

## Load images

In [ ]:
images = []
for i in mio.import_images('/vol/atlas/databases/lfpw/trainset/', 
                           verbose=True, max_images=10):
    if i.n_channels == 3:
        i = i.crop_to_landmarks_proportion(0.5)
        i = i.rescale_landmarks_to_diagonal_range(200)
        labeller(i, 'PTS', ibug_face_66)
        #i = fast_dsift(i)
        images.append(i)

In [ ]:
visualize_images(images)

In [ ]:
shapes = [i.landmarks['ibug_face_66'].lms for i in images]

In [ ]:
patch_experts = build_lsvm_ensemble(
    images, 
    shapes, 
    patch_size=(16, 16), 
    context_size=2,
    C=100)

In [ ]:
filters = [pe.frequency_filter_image for pe in patch_experts]

In [ ]:
visualize_images(filters)

In [ ]:
i = images[0]
patch_experts[9].predict(i).view(cmap_name='jet')

In [ ]:
%timeit patch_experts[46].predict(i)

In [ ]:
from pyfftw.interfaces.numpy_fft import ifft2, fft2

In [ ]:
a = np.random.randn(8, 32, 32).astype(np.float32)
b = np.random.randn(66, 8, 32, 32).astype(np.complex64)

In [ ]:
%timeit c = ifft2(np.sum(fft2(a, threads=1).astype(np.complex64) * b, axis=1), threads=1)

In [ ]:
%timeit ifft2(sum(b, axis=1))


In [ ]:
c = ifft2(np.sum(fft2(a) * b, axis=1))
c.shape